<div style="direction:rtl;line-height:500%;">
	<font face="B Zar", size=5px>
        <h2>هوش مصنوعی، تمرین دوم پیاده‌سازی</h2>
        احمد سلیمی - ۹۷۱۰۶۰۱۳
    </font>
</div>

<div style="direction:rtl;line-height:300%;">
	<font face="B Zar", size=5px>
        ابتدا کتابخانه‌های مورد نیاز را <code>import</code> می‌کنیم.
    </font>
</div>

In [1]:
import numpy as np
import pandas as pd
import json

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>JsonEncoder</code></h4>
        <br>
        این کلاس برای تبدیل آبجکت‌های موجود به فرمت Json
        می‌باشد.
    </font>
</div>

In [2]:
class JsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, TerminalNode):
            return obj.outcome
        if isinstance(obj, DecisionNode):
            return {
                "Feature": obj.feature, 
                "Information gain": obj.information_gain,
                "Entropy": obj.entropy,
                "Children": {repr(condition): obj.children[condition] for condition in obj.children}
                }
        if isinstance(obj, Condition):
            return obj.to_json()
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>Condition</code></h4>
        <br>
        یک کلاس abstract
        است که برای نگهداری و اعمال یک شرط روی feature
        ها می‌باشد.
    </font>
</div>

In [3]:
class Condition:

    def check(self, value):
        raise NotImplementedError()
    
    def to_json(self):
        return repr(self)

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>ExactCondition</code></h4>
        <br>
        این کلاس که از کلاس
        <code>Condition</code>
        ارث‌بری می‌کند، برای نگهداری شرط روی feature
        های گسسته می‌باشد.
    </font>
</div>

In [4]:
class ExactCondition(Condition):

    def __init__(self, value):
        self.value = value
    
    def check(self, value):
        return self.value == value
    
    def to_json(self):
        return self.value

    def __repr__(self):
        return str(self.value)
    
    def __hash__(self):
        return hash(repr(self))

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            self.value == other.value
        )

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>RangeCondition</code></h4>
        <br>
        این کلاس که از کلاس
        <code>Condition</code>
        ارث‌بری می‌کند، برای نگهداری شرط روی feature
        های پیوسته، و بصورت بازه‌ای می‌باشد.
    </font>
</div>

In [5]:
class RangeCondition(Condition):

    def __init__(self, lower_bound=-float('inf'), upper_bound=float('inf')):
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.check = np.vectorize(self.single_check)
    
    def single_check(self, x):
        return x <= self.upper_bound and x >= self.lower_bound
    
    def __repr__(self):
        return f'{self.lower_bound}-{self.upper_bound}'
    
    def __hash__(self):
        return hash(repr(self))

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            self.lower_bound == other.lower_bound and
            self.upper_bound == other.upper_bound
        )

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>Node</code></h4>
        <br>
        یک کلاس abstract
        برای بازنمایی یک راس از درخت تصمیم است.
    </font>
</div>

In [6]:
class Node:

    def search(self, row):
        raise NotImplementedError()
    
    def __repr__(self):
        return json.dumps(self, cls=JsonEncoder, indent=4)

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>DecisionNode</code></h4>
        <br>
        این کلاس که از کلاس
        <code>Node</code>
        ارث‌بری می‌کند، برای بازنمایی یک راس تصمیم از درخت تصمیم است، که حاوی feature،
        information_gain،
        entropy
        و لیست فرزندان است.
    </font>
</div>

In [7]:
class DecisionNode(Node):

    def __init__(self, feature, information_gain, entropy):
        self.feature = feature
        self.children = {}
        self.information_gain = information_gain
        self.entropy = entropy
    
    def search(self, row):
        for condition in self.children:
            if condition.check(row[self.feature]):
                return self.children[condition].search(row)
        raise Exception(f'feature {self.feature} with value {row[self.feature]} could not be recognized.')

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>TerminalNode</code></h4>
        <br>
        این کلاس که از کلاس
        <code>Node</code>
        ارث‌بری می‌کند، برای بازنمایی یک راس نهایی از درخت تصمیم است، که حاوی outcome
        است.
    </font>
</div>

In [8]:
class TerminalNode(Node):

    def __init__(self, outcome):
        self.outcome = outcome
    
    def search(self, row):
        return self.outcome

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>ConditionContainer</code></h4>
        <br>
        این کلاس در بر دارنده‌ی اطلاعات مربوط به شرط‌ها و split
        های یک feature
        است.
    </font>
</div>

In [9]:
class ConditionContainer:

    def __init__(self, feature_name, conditions, information_gain, entropy):
        self.feature_name = feature_name
        self.conditions = conditions
        self.information_gain = information_gain
        self.entropy = entropy
    
    def __lt__(self, other):
        return self.information_gain < other.information_gain

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>FeatureHandler</code></h4>
        <br>
        یک کلاس abstract
        که وظیفه‌اش مدیریت یک feature
        و بررسی اهمیت آن feature
        در یک مرحله از یادگیری می‌باشد.
    </font>
</div>

In [10]:
class FeatureHandler:

    def __init__(self, name):
        self.name = name
    
    def prepare_conditions(self, X, y):
        raise NotImplementedError()
    
    def importance(self, conditions, X, y):
        entropy = self.entropy(y.sum() / len(y))
        return ConditionContainer(
            self.name,
            conditions, 
            entropy - self.remainder(conditions, X, y),
            entropy
        )

    def remainder(self, conditions, X, y):
        result = 0
        for condition in conditions:
            filtered_y = y[condition.check(X[self.name])]
            p_k = filtered_y.sum()
            if p_k == 0:
                continue
            result += self.entropy(p_k / len(filtered_y)) * len(filtered_y) / len(y)
        
        return result

    def entropy(self, prob):
        if prob == 0 or prob == 1:
            return 0
        return -(prob * np.log2(prob) + (1 - prob) * np.log2(1 - prob))

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>ExactFeatureHandler</code></h4>
        <br>
        این کلاس که از کلاس
        <code>FeatureHandler</code>
        ارث‌بری می‌کند، برای مدیریت feature
        های گسسته استفاده می‌شود.
    </font>
</div>

In [11]:
class ExactFeatureHandler(FeatureHandler):

    def __init__(self, name, values):
        super().__init__(name)
        self.conditions = [ExactCondition(value) for value in values.unique()]
    
    def prepare_conditions(self, X, y):
        return self.importance(self.conditions, X, y)

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>RangeFeatureHandler</code></h4>
        <br>
        این کلاس که از کلاس
        <code>FeatureHandler</code>
        ارث‌بری می‌کند، برای مدیریت feature
        های پیوسته استفاده می‌شود.
        این کلاس، با توجه $k$
        مقدار ممکن برای یک فیچر، $k - 1$
        نقطه‌ی میانی آن مقادیر را انتخاب کرده، و به ازای هر کدام از آن‌ها، بازه‌ای برای مقادیر کوچکتر، و بازه‌ای برای مقادیر بزرگتر از آن مقدار می‌سازد.
        در نهایت، با توجه به داده‌های ورودی، اهمیت هر یک از این $k - 1$
        بازه را سنجیده و مهم‌ترین آن‌ها را در قالب یک ConditionContainer
        بر می‌گرداند.
    </font>
</div>

In [12]:
class RangeFeatureHandler(FeatureHandler):

    def __init__(self, name, values):
        super().__init__(name)
        self.create_condition_set(sorted(values.unique()))
    
    def create_condition_set(self, sorted_points):
        split_points = [(sorted_points[i] + sorted_points[i + 1]) / 2 for i in range(len(sorted_points) - 1)]
        self.condition_set = [self.split(point) for point in split_points]
    
    def split(self, point):
        return [RangeCondition(upper_bound=point), RangeCondition(lower_bound=point)]
    
    def prepare_conditions(self, X, y):
        return max([self.importance(conditions, X, y) for conditions in self.condition_set])

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>کلاس<code>DecisionTreeBinaryClassifier</code></h4>
        <br>
        این کلاس، در بر گیرنده‌ی الگوریتم یادگیری و ساخت درخت تصمیم برای Binary Classification
        است.
        <br>
        <b>پارامترها</b>
        <ol>
            <li>حداکثر عمق درخت</li>
        </ol>
        <b>مراحل</b>
        <ol>
            <li>
                <b>
                    ساخت FeatureHandler
                    ها با توجه به feature
                    ها:
                </b>
                نکته‌ی قابل توجه آن است که در ابتدا باید یک دیکشنری که نوع فیچرها از نظر پیوسته یا گسسته بودن را تعیین می‌کند، بعنوان ورودی داده‌شود.
                حال با توجه به نوع فیچر، FeatureHandler
                مربوط به هر فیچر ساخته می‌شود.
            </li>
            <li>
                <b>یادگیری درخت تصمیم:</b>
                در این مرحله، درخت تصمیم در تابع
                <br>
                <code>create_tree(X, y, current_features, parent_y, height)</code>
                <br>
                بصورت بازگشتی ساخته می‌شود.
            </li>
        </ol>
    </font>
</div>

In [13]:
discrete = "discrete"
continuous = "continuous"

class DecisionTreeBinaryClassifier:

    def __init__(self, max_depth=5):
        self.feature_handlers = {}
        self.tree = None
        self.height = 0
        self.max_depth = max_depth
    
    def generate_feature_handlers(self, X:pd.DataFrame, feature_types:dict):
        for feature in X.columns:
            if feature_types[feature] == continuous:
                self.feature_handlers[feature] = RangeFeatureHandler(feature, X[feature])
            else:
                self.feature_handlers[feature] = ExactFeatureHandler(feature, X[feature])

    def plurality_value(self, y):
        return 1 if y.sum() / len(y) >= 0.5 else 0
    
    def create_tree(self, X:pd.DataFrame, y:pd.Series, current_features:pd.Series, parent_y:pd.Series=None, height=0):
        self.height = max(self.height, height)
        if X.empty: # no remaining sample row
            return TerminalNode(self.plurality_value(parent_y))
        if y.nunique() == 1: # all rows has a same outcome
            return TerminalNode(y.iloc[0])
        if not any(current_features) or self.height > self.max_depth: # all features are checked or max height reached
            return TerminalNode(self.plurality_value(y))
        
        # choose best feature and split:
        condition_container = max(map(lambda feature: self.feature_handlers[feature].prepare_conditions(X, y), current_features))
        
        tree = DecisionNode(
            condition_container.feature_name, 
            condition_container.information_gain, 
            condition_container.entropy)
        
        new_features = current_features.drop(tree.feature)

        for condition in condition_container.conditions:
            filter = condition.check(X[tree.feature])
            subtree = self.create_tree(X[filter], y[filter], new_features, y, height+1)
            subtree.condition = condition
            tree.children[condition] = subtree

        return tree
    
    def fit(self, X:pd.DataFrame, y:pd.Series, feature_types:dict):
        self.generate_feature_handlers(X, feature_types)
        self.tree = self.create_tree(X, y, X.columns)
        return self.tree
    
    def predict(self, X_test:pd.DataFrame):
        return X_test.apply(lambda row: self.tree.search(row), axis=1)

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>تابع<code>calculate_accuracy</code></h4>
        <br>
        این تابع، مقادیر تخمین زده شده را با مقادیر واقعی مقایسه کرده، و میزان صحت تخمین را برمی‌گرداند.
    </font>
</div>

In [14]:
def calculate_accuracy(y_pred, y_test):
    return (y_pred == y_test).sum() / len(y_pred)

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        ابتدا، داده‌ی مربوط یه رستوران را آزمایش می‌کنیم.
    </font>
</div>

In [15]:
sample_data = pd.read_csv('sample_tree.csv')
sample_data

,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Outcome
0,1,0,0,1,Some,3,0,1,French,0-10,1
1,1,0,0,1,Full,1,0,0,Thai,30-60,0
2,0,1,0,0,Some,1,0,0,Burger,0-10,1
3,1,0,1,1,Full,1,1,0,Thai,10-30,1
4,1,0,1,0,Full,3,0,1,French,>60,0
5,0,1,0,1,Some,2,1,1,Italian,0-10,1
6,0,1,0,0,None,1,1,0,Burger,0-10,0
7,0,0,0,1,Some,2,1,1,Thai,0-10,1
8,0,1,1,0,Full,1,1,0,Burger,>60,0
9,1,1,1,1,Full,3,0,1,Italian,10-30,0


<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        feature
        ها را از Outcome
        جدا می‌کنیم.
    </font>
</div>

In [16]:
X, y = sample_data.drop('Outcome', axis=1), sample_data['Outcome']

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        سپس نوع هر یک از feature
        ها را تعیین می‌کنیم.
    </font>
</div>

In [17]:
feature_types = {feature: discrete for feature in X.columns}
feature_types

{'Alt': 'discrete',
 'Bar': 'discrete',
 'Fri': 'discrete',
 'Hun': 'discrete',
 'Pat': 'discrete',
 'Price': 'discrete',
 'Rain': 'discrete',
 'Res': 'discrete',
 'Type': 'discrete',
 'Est': 'discrete'}

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        یک instance
        از کلاس DecisionTreeBinaryClassifier
        می‌سازیم.
    </font>
</div>

In [18]:
sample_decision = DecisionTreeBinaryClassifier()

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        یادگیری روی داده‌ها انجام شده و درخت چاپ می‌شود. همانطور که مشاهده می‌شود، درخت ساخته شده، دقیقا مشابه درختی است که در اسلایدها نیز بود.
    </font>
</div>

In [19]:
sample_decision.fit(X, y, feature_types)

{
    "Feature": "Pat",
    "Information gain": 0.5408520829727552,
    "Entropy": 1.0,
    "Children": {
        "Some": 1,
        "Full": {
            "Feature": "Hun",
            "Information gain": 0.2516291673878229,
            "Entropy": 0.9182958340544896,
            "Children": {
                "1": {
                    "Feature": "Type",
                    "Information gain": 0.5,
                    "Entropy": 1.0,
                    "Children": {
                        "French": 1,
                        "Thai": {
                            "Feature": "Fri",
                            "Information gain": 1.0,
                            "Entropy": 1.0,
                            "Children": {
                                "0": 0,
                                "1": 1
                            }
                        },
                        "Burger": 1,
                        "Italian": 0
                    }
                },
                "0": 0

In [20]:
sample_decision.height

4

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        صحت روی داده‌ها را چاپ می‌کنیم. همانطور که مشخص است، درخت تصمیم تمام داده‌ها را به درستی تشخیص می‌دهد.
    </font>
</div>

In [21]:
prediction = sample_decision.predict(X)
calculate_accuracy(prediction, y)

1.0

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        حال داده‌های مربوط به دیابت را آزمایش می‌کنیم.
    </font>
</div>

In [22]:
diabetes = pd.read_csv('diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [23]:
diabetes_feature_types = {feature: continuous for feature in diabetes.columns[diabetes.columns != 'Outcome']}
diabetes_feature_types

{'Pregnancies': 'continuous',
 'Glucose': 'continuous',
 'BloodPressure': 'continuous',
 'SkinThickness': 'continuous',
 'Insulin': 'continuous',
 'BMI': 'continuous',
 'DiabetesPedigreeFunction': 'continuous',
 'Age': 'continuous'}

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        ۲۰ درصد از داده‌ها را برای تست جدا می‌کنیم.
    </font>
</div>

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome', axis=1), diabetes['Outcome'], test_size=0.99)

In [49]:
diabetes_classifier = DecisionTreeBinaryClassifier()

In [50]:
diabetes_classifier.fit(X_train, y_train, diabetes_feature_types)

{
    "Feature": "BMI",
    "Information gain": 0.4695652111147069,
    "Entropy": 0.863120568566631,
    "Children": {
        "-inf-35.150000000000006": {
            "Feature": "Glucose",
            "Information gain": 0.9182958340544896,
            "Entropy": 0.9182958340544896,
            "Children": {
                "-inf-136.0": 1,
                "136.0-inf": 0
            }
        },
        "35.150000000000006-inf": 0
    }
}

In [51]:
diabetes_classifier.height

2

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        همانطور که مشاهده می‌شود، دقت درخت روی داده‌های train
        و test
        تقریبا یکسان است و این یعنی مدل overfit
        نشده است. اما هرگونه بهینه‌سازی، نتوانست دقت را از این بالاتر ببرد.
        حدس من این است که با توجه به این که بخش زیادی از داده، ناقص است و در برخی از داده‌ها مقدار بعضی از فیچر ها صفر است، داده نیاز به تمیزکاری دارد.
        مثلا می‌توان مقادیر نامشخص را با میانگین کل داده‌ها جایگزین کرد.
    </font>
</div>

In [52]:
train_prediction = diabetes_classifier.predict(X_train)
f'train accuracy: {calculate_accuracy(train_prediction, y_train)}'

'train accuracy: 1.0'

In [53]:
test_prediction = diabetes_classifier.predict(X_test)
f'test accuracy: {calculate_accuracy(test_prediction, y_test)}'

'test accuracy: 0.3363994743758213'

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        به جهت مقایسه، داده‌ها روی دو الگوریتم، با استفاده از کتابخانه‌های آماده نیز آموزش داده‌شدند که در ادامه مشاهده می‌کنید:
        <h4>کلاس DecisionTreeClassifier از کتابخانه‌ی sci-kit leaen</h4>
    </font>
</div>

In [34]:
from sklearn.tree import DecisionTreeClassifier
sk_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [35]:
sk_classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        همانطور که مشاهده می‌شود، دقت الگوریتم آن‌چنان بیشتر از درخت تصمیم پیاده‌سازی شده نیست.
    </font>
</div>

In [36]:
train_prediction = sk_classifier.predict(X_train)
f'train accuracy: {calculate_accuracy(train_prediction, y_train)}'

'train accuracy: 0.8306188925081434'

In [37]:
test_prediction = sk_classifier.predict(X_test)
f'sklearn test accuracy: {calculate_accuracy(test_prediction, y_test)}'

'sklearn test accuracy: 0.7662337662337663'

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        <h4>شبکه عصبی Sequential از کتابخانه‌ی keras و tensorflow</h4>
        <br>
        شبکه عصبی طراحی شده، دارای یک لایه‌ی Dense
        ورودی، یک لایه‌ی hidden
        و یک لایه خروجی می‌باشد که در ادامه مشاهده می‌کنید.
    </font>
</div>

In [48]:
import keras
from keras.models import Sequential #used to initialize the NN
from keras.layers import Dense  #used to build the hidden Layers
from keras.layers import Dropout

Using TensorFlow backend.


In [49]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome', axis=1).values, diabetes['Outcome'].values, test_size=0.2)

In [51]:
diabetes_NN = Sequential()
diabetes_NN.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 8))
diabetes_NN.add(Dropout(rate=0.1))

diabetes_NN.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
diabetes_NN.add(Dropout(rate=0.1))

diabetes_NN.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
diabetes_NN.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
diabetes_NN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                288       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,377
Trainable params: 1,377
Non-trainable params: 0
_________________________________________________________________


In [56]:
_ = diabetes_NN.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1)

Train on 552 samples, validate on 62 samples
Epoch 1/100
552/552 [==============================] - 0s 179us/step - loss: 0.5079 - accuracy: 0.7337 - val_loss: 0.4931 - val_accuracy: 0.7903
Epoch 2/100
552/552 [==============================] - 0s 291us/step - loss: 0.4855 - accuracy: 0.7609 - val_loss: 0.4896 - val_accuracy: 0.7419
Epoch 3/100
552/552 [==============================] - 0s 246us/step - loss: 0.4657 - accuracy: 0.7736 - val_loss: 0.4975 - val_accuracy: 0.7903
Epoch 4/100
552/552 [==============================] - 0s 226us/step - loss: 0.4603 - accuracy: 0.7826 - val_loss: 0.4968 - val_accuracy: 0.7742
Epoch 5/100
552/552 [==============================] - 0s 244us/step - loss: 0.4418 - accuracy: 0.7862 - val_loss: 0.4942 - val_accuracy: 0.7581
Epoch 6/100
552/552 [==============================] - 0s 208us/step - loss: 0.4860 - accuracy: 0.7627 - val_loss: 0.4997 - val_accuracy: 0.7419
Epoch 7/100
552/552 [==============================] - 0s 267us/step - loss: 0.4878 -

Epoch 57/100
552/552 [==============================] - 0s 179us/step - loss: 0.5014 - accuracy: 0.7609 - val_loss: 0.4780 - val_accuracy: 0.7419
Epoch 58/100
552/552 [==============================] - 0s 187us/step - loss: 0.4657 - accuracy: 0.7772 - val_loss: 0.5032 - val_accuracy: 0.7097
Epoch 59/100
552/552 [==============================] - 0s 192us/step - loss: 0.4730 - accuracy: 0.7790 - val_loss: 0.4969 - val_accuracy: 0.7742
Epoch 60/100
552/552 [==============================] - 0s 173us/step - loss: 0.4703 - accuracy: 0.7808 - val_loss: 0.4810 - val_accuracy: 0.8065
Epoch 61/100
552/552 [==============================] - 0s 146us/step - loss: 0.4669 - accuracy: 0.7790 - val_loss: 0.4832 - val_accuracy: 0.7742
Epoch 62/100
552/552 [==============================] - 0s 213us/step - loss: 0.4459 - accuracy: 0.8043 - val_loss: 0.4854 - val_accuracy: 0.7742
Epoch 63/100
552/552 [==============================] - 0s 197us/step - loss: 0.4563 - accuracy: 0.7699 - val_loss: 0.4964 -

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        همانطور که مشاهده می‌شود، دقت مدل آموزش داده شده توسط شبکه عصبی نیز آن‌چنان بیشتر از درخت تصمیم پیاده‌سازی شده نیست.
    </font>
</div>

In [57]:
train_prediction = diabetes_NN.predict(X_train)
train_prediction = (train_prediction.reshape((len(train_prediction),)) > 0.5)
f'train accuracy: {calculate_accuracy(train_prediction, y_train)}'

'train accuracy: 0.8045602605863192'

In [58]:
test_prediction = diabetes_NN.predict(X_test)
test_prediction = (test_prediction.reshape((len(test_prediction),)) > 0.5)
f'test accuracy: {calculate_accuracy(test_prediction, y_test)}'

'test accuracy: 0.7727272727272727'

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        برای ارزیابی بیشتر الگوریتم پیاده‌سازی شده، درخت تصمیم را روی یک داده‌ی دیگر نیز آزمایش کردم.
        این داده مربوط به تشخیص خوش‌خیم یا بدخیم بودن سرطان سینه است که از کتابخانه‌ی sci-kit learn
        بارگیری شده است. این مجموعه داده، شامل ۵۶۹ نمونه، و ۳۰ feature
        است.
    </font>
</div>

In [59]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
cancer_X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
cancer_y = pd.Series(cancer.target)
print(cancer_X.shape)
cancer_X.head()

(569, 30)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [60]:
cancer_feature_types = {feature: continuous for feature in cancer.feature_names}
cancer_feature_types

{'mean radius': 'continuous',
 'mean texture': 'continuous',
 'mean perimeter': 'continuous',
 'mean area': 'continuous',
 'mean smoothness': 'continuous',
 'mean compactness': 'continuous',
 'mean concavity': 'continuous',
 'mean concave points': 'continuous',
 'mean symmetry': 'continuous',
 'mean fractal dimension': 'continuous',
 'radius error': 'continuous',
 'texture error': 'continuous',
 'perimeter error': 'continuous',
 'area error': 'continuous',
 'smoothness error': 'continuous',
 'compactness error': 'continuous',
 'concavity error': 'continuous',
 'concave points error': 'continuous',
 'symmetry error': 'continuous',
 'fractal dimension error': 'continuous',
 'worst radius': 'continuous',
 'worst texture': 'continuous',
 'worst perimeter': 'continuous',
 'worst area': 'continuous',
 'worst smoothness': 'continuous',
 'worst compactness': 'continuous',
 'worst concavity': 'continuous',
 'worst concave points': 'continuous',
 'worst symmetry': 'continuous',
 'worst fractal d

In [61]:
X_train, X_test, y_train, y_test = train_test_split(cancer_X, cancer_y, test_size=0.2)

In [62]:
cancer_classifier = DecisionTreeBinaryClassifier()

In [64]:
cancer_classifier.fit(X_train, y_train, cancer_feature_types)

{
    "Feature": "worst area",
    "Information gain": 0.5710364609604932,
    "Entropy": 0.9517626756348311,
    "Children": {
        "-inf-874.85": {
            "Feature": "worst concave points",
            "Information gain": 0.1626179789602885,
            "Entropy": 0.4213582670988372,
            "Children": {
                "-inf-0.131": {
                    "Feature": "area error",
                    "Information gain": 0.03221869686929066,
                    "Entropy": 0.1366813204226173,
                    "Children": {
                        "-inf-38.415000000000006": {
                            "Feature": "worst texture",
                            "Information gain": 0.023119479722836712,
                            "Entropy": 0.06789640139994144,
                            "Children": {
                                "-inf-30.08": 1,
                                "30.08-inf": {
                                    "Feature": "texture error",
               

In [65]:
cancer_classifier.height

6

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        در این مثال، مشاهده می‌شود که مقدار دقت، بسیار بیشتر از داده‌های دیابت است، و این، مسئله‌ی وجود نقص در داده‌ی دیابت را قوی‌تر می‌کند.
    </font>
</div>

In [66]:
train_prediction = cancer_classifier.predict(X_train)
f'train accuracy: {calculate_accuracy(train_prediction, y_train)}'

'train accuracy: 0.9274725274725275'

In [67]:
test_prediction = cancer_classifier.predict(X_test)
f'train accuracy: {calculate_accuracy(test_prediction, y_test)}'

'train accuracy: 0.9035087719298246'

<div style="direction:rtl;line-height:300%;text-align:justify;">
	<font face="B Zar", size=5px>
        در نتیجه‌ی آزمایش‌های انجام شده، بنظرم اگر قبل از آموزش مدل، باید داده‌ها را حتما تمیزکاری کرد.
        <br>
        برای جلوگیری از overfitting،
        عمق درخت را محدود کردم، و تا حد خیلی خوبی جواب داد.
    </font>
</div>